In [ ]:
import holoviews as hv; hv.extension("bokeh", logo=False)
import panel as pn; pn.extension()
import networkx as nx

hv.opts.defaults(hv.opts.Image(cmap='gray', xaxis=None, yaxis=None, width=250,height=300))

import numpy as np
import os
from matplotlib.image import imread

In [ ]:
class PointLabeler:
    def __init__(self):
        self.reset()

    def reset(self):
        self.G      = nx.Graph()
        self.E      = {}
        self.pos    = {}
        self.text   = {}
        self.color  = {}

    def _add( self, point_id, point_pos, lbl_id, lbl_pos, text, at_start=False, color='black' ):
        self.E[(point_id,lbl_id)]     = text
        self.color[(point_id,lbl_id)] = color
        self.pos[point_id]            = np.array(point_pos)
        self.pos[lbl_id]              = np.array(lbl_pos)
        self.text[point_id if at_start else lbl_id] = {"text" : text, "color": color}

    def _add_lbl( self, lbl_id, point_id, delta, text, color="black" ):
        #print( f"adding {delta} for {lbl_id} to current {point_id} at {self.pos[point_id]}")
        self.G.add_node(lbl_id)
        self.pos[lbl_id]  = self.pos[point_id] + delta
        self.text[lbl_id] = {"text" : text, "color": color}

    def add( self, point_pos, lbl_pos, text, at_start=False, offset=None, color='black'):
        cur = len( self.pos )+1
        if offset is None:
            self._add( cur, point_pos, cur+1, lbl_pos, text, at_start, color )
        else:
            self._add( cur, point_pos, cur+1, lbl_pos, "", at_start, color )
            diff     = self.pos[cur+1] - self.pos[cur]
            diff_pos = offset * diff / np.linalg.norm( diff)
            if at_start:
                self._add_lbl( cur+2, cur, -diff_pos, text, color)
            else:
                self._add_lbl( cur+2, cur+1, diff_pos, text, color)
                
    def graph(self):
        _ = [self.text.__setitem__(k,{"text": "", "color": "black"}) for k in self.pos.keys() if self.text.get(k,None) is None]

        #self.G.add_edges_from(self.E.keys())
        for e in self.E.keys():
            self.G.add_edge( *e, color=self.color[e])
        nx.set_node_attributes( self.G, self.text )
        graph  = hv.Graph.from_networkx(self.G, self.pos)\
                   .opts( directed=True, node_alpha=0, arrowhead_length=0.5, edge_color="color")
        lbls   = hv.Labels( graph.nodes, ['x', 'y'], ["text","color"]).opts(text_color='color')
        return graph,lbls

    def show(self):
        graph,lbls = self.graph()
        return graph*lbls

    def _dbg(self):
        print("Pos")
        for i in self.pos.keys():
            print( f".  {i, self.pos[i]}")
        print("Edge")
        for i in self.E.keys():
            print( f".  {i, self.E[i]}")
        print("Text")
        for i in self.text.keys():
            print( f".  {i, self.text[i]}")

In [ ]:
A = imread(os.path.join('..','DATA','dog.jpg'))
X = np.mean(A, -1); # Convert RGB to grayscale
hv.Image(X)

In [ ]:
U, S, VT = np.linalg.svd(X,full_matrices=False)
S = np.diag(S)
approx = [4,19,99]
h = []
j = 0
for r in approx:
    # Construct approximate image
    Xapprox = U[:,:r] @ S[0:r,:r] @ VT[:r,:]
    h.append( hv.Image(Xapprox).opts(title=f"r = {r+1}"))
pn.Row( *h )

In [ ]:
cs = np.cumsum(np.diag(S))/np.sum(np.diag(S))

sc_pl = PointLabeler()
for k in approx:
    sc_pl.add( (400,cs[k]), (k,cs[k]), f"r = {k+1}", offset=25,  at_start=True, color='red')


pn.Column(
    pn.pane.Markdown("# Singular Values, Red Markers show the Approximations Used"),
    pn.Row(
        hv.Curve( np.diag(S)).opts(logy=True, width=400, title="Singular Values")*\
        hv.Scatter((approx, [S[k,k] for k in approx])).opts(color="red",size=4),
        hv.Curve( cs ).opts(width = 400, title="Singular Values: Cumulative Sum (Scaled)")*\
        hv.Scatter((approx, [cs[k] for k in approx])).opts(color="red",size=4)*\
        sc_pl.show().opts(hv.opts.Labels(text_align="left"),
                          hv.opts.Graph(arrowhead_length=0.0001)
                         )
    )
)

**Dominant Correlations** of $X = U_r \Sigma_r V^t_r$
* $X^t X = V_r \Sigma_r^2 V^t_r$
* $X X^t = U_r \Sigma_r^2 U^t_r$

In [ ]:
pn.Row( hv.Image( X @ X.T).opts(title='Col Correlations: X X.T'),
        hv.Image( X      ).opts(title='Image X'),
        hv.Image( X.T @ X).opts(title='Row Correlations: X.T X') )